In [5]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from statsmodels.tsa.api import VAR
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Configurações gerais
plt.rcParams.update({'figure.max_open_warning': 0})

# Diretório para salvar as figuras
output_dir = 'figuras_var'
os.makedirs(output_dir, exist_ok=True)

# Carregando o DataFrame
df = pd.read_csv('pca_reduced_data.csv')

# Selecionando apenas as colunas pares (partes reais)
data = df.iloc[:, ::2]

# Transpor para ter as séries temporais nas colunas
data = data.T

# Divisão em treinamento e teste (com continuidade temporal)
train_size = int(len(data) * 0.75)
train_data = data.iloc[:train_size]
test_data = data.iloc[train_size:]

# Ajustar o modelo VAR nos dados de treinamento
model_var = VAR(train_data)
# Selecionar automaticamente a ordem do modelo (número de defasagens) usando o critério AIC
results_aic = model_var.select_order(maxlags=15)
selected_lag = results_aic.aic
print(f"Ordem do modelo VAR selecionada (AIC): {selected_lag}")
model_fitted = model_var.fit(selected_lag)

# Fazer previsões no conjunto de teste
lag_order = model_fitted.k_ar
print(f"Ordem do modelo ajustado: {lag_order}")

# Preparar os dados para a previsão
input_data = train_data.values[-lag_order:]

# Número de previsões a serem feitas
num_predictions = len(test_data)

# Fazer previsões
forecast_results = model_fitted.forecast(y=input_data, steps=num_predictions)
forecast_index = test_data.index

# Converter previsões em DataFrame
predictions = pd.DataFrame(forecast_results, index=forecast_index, columns=data.columns)

# Calcular métricas de erro
mae = mean_absolute_error(test_data, predictions)
mse = mean_squared_error(test_data, predictions)
print(f'MAE: {mae:.4f}, MSE: {mse:.4f}')

# Função para calcular NMSE por característica
def calculate_normalized_mse(actual, predicted):
    num_features = actual.shape[1]
    nmse = np.zeros(num_features)
    for c in range(num_features):
        mse = mean_squared_error(actual.iloc[:, c], predicted.iloc[:, c])
        variance = np.var(actual.iloc[:, c])
        if variance != 0:
            nmse[c] = mse / variance
        else:
            nmse[c] = np.nan
    return nmse

# Calcular NMSE para cada característica
nmse = calculate_normalized_mse(test_data, predictions)
average_nmse = np.nanmean(nmse)
print(f"Média do NMSE: {average_nmse:.4f}")

# Plotar NMSE por característica
features = np.arange(1, data.shape[1] + 1)
plt.figure(figsize=(12, 6), dpi=400)
plt.bar(features, nmse, width=0.6, color='blue')
plt.xlabel('Índice da Característica')
plt.ylabel('NMSE')
plt.title('NMSE por Característica - VAR')
plt.grid(True)
# Rotacionar os rótulos do eixo x
plt.xticks(features, [f'Feat {int(f)}' for f in features], rotation=45, ha='right')
# Reduzir o número de ticks no eixo x
plt.gca().xaxis.set_major_locator(plt.MaxNLocator(20))  # Limitar a 20 ticks ou ajuste conforme necessário
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'nmse_por_caracteristica_var.png'), dpi=400)
plt.close()

# Plotar predições vs valores reais para algumas características em subplots
num_features_to_plot = 4  # Número de características a serem plotadas
features_to_plot = np.random.choice(data.columns, num_features_to_plot, replace=False)

fig, axs = plt.subplots(num_features_to_plot, 1, figsize=(12, 3 * num_features_to_plot), dpi=400)
for idx, feature in enumerate(features_to_plot):
    axs[idx].plot(test_data.index, test_data[feature], label=f'Valor Real - {feature}')
    axs[idx].plot(predictions.index, predictions[feature], label=f'Previsão - {feature}', linestyle='--')
    axs[idx].set_title(f'Previsão vs Valor Real - {feature} - VAR')
    axs[idx].set_xlabel('Amostra')
    axs[idx].set_ylabel('Valor')
    axs[idx].legend()
    axs[idx].grid(True)
    # Rotacionar os rótulos do eixo x para evitar sobreposição
    axs[idx].tick_params(axis='x', rotation=45)
    # Reduzir o número de ticks no eixo x
    axs[idx].xaxis.set_major_locator(plt.MaxNLocator(10))
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'previsao_vs_real_var_subplots.png'), dpi=400)
plt.close()


c:\Users\FilipeAraujoXimenes\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:559: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  _index = to_datetime(index)
c:\Users\FilipeAraujoXimenes\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


Ordem do modelo VAR selecionada (AIC): 15
Ordem do modelo ajustado: 15
MAE: 0.0021, MSE: 0.0001
Média do NMSE: 0.0000
